In [1]:
%load_ext autoreload
%autoreload 2
%cd ../../

import sys
import os
sys.path.append(f"{os.getcwd()}/src")

/Users/tccuong1404/Documents/Projects/visuelle-sales-forecasting


/Users/tccuong1404/Documents/Projects/visuelle-sales-forecasting/.venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [150]:
import pandas as pd
import seaborn as sns
from datetime import datetime
from datetime import timedelta
from matplotlib import pyplot as plt
from models import VisuelleDataset

In [158]:
visuelle = VisuelleDataset(data_dir="dataset/")

2024-02-01 21:46:31.165 | INFO     | models.visuelle_dataset:__init__:31 - ====== DATASET ======
2024-02-01 21:46:31.165 | INFO     | models.visuelle_dataset:__init__:32 - Train path: dataset/train.csv
2024-02-01 21:46:31.166 | INFO     | models.visuelle_dataset:__init__:33 - Test path: dataset/test.csv
2024-02-01 21:46:31.166 | INFO     | models.visuelle_dataset:__init__:34 - Category path: dataset/category_labels.pt
2024-02-01 21:46:31.166 | INFO     | models.visuelle_dataset:__init__:35 - Color path: dataset/color_labels.pt
2024-02-01 21:46:31.166 | INFO     | models.visuelle_dataset:__init__:36 - Fabric path: dataset/fabric_labels.pt


In [142]:
def transform_to_daily_sales(df: pd.DataFrame, attr: str, num_weeks: int=12):
    min_date = df["release_date"].min()
    max_date = df["release_date"].max() + timedelta(days=7 * num_weeks)
    attr_ls = list(df[attr].unique())
    print(min_date, max_date)
    num_days = int((max_date - min_date).days)
    df_dates = pd.DataFrame(range(num_days), columns=["days"])
    df_dates[attr] = [attr_ls] * len(df_dates)
    df_dates["anchor"] = min_date
    df_dates["delta"] = pd.to_timedelta(df_dates["days"], unit="days")
    df_dates["date"] = df_dates["anchor"] + df_dates["delta"]
    df_dates = df_dates.drop(columns=["anchor", "delta", "days"])
    df_dates = df_dates.explode(column=attr)

    tmp = pd.DataFrame(range(7), columns=["day_of_week"])
    df = pd.merge(df, tmp, how="cross")
    df = df.melt(id_vars=[attr, "release_date", "day_of_week"], value_vars=df.columns[:12])
    df["delta"] = (df["variable"].astype(int) * 7) + df["day_of_week"]
    df["delta"] = pd.to_timedelta(df["delta"], unit="days")
    df["date"] = df["release_date"] + df["delta"]
    df["value"] = df["value"] / 7
    df = df.rename(columns={
        "value": "sales"
    })
    df = df.drop(columns=["variable", "release_date", "delta", "day_of_week"])
    df = pd.merge(df_dates, df, on=[attr, "date"], how="left")
    df = df.fillna({"sales": 0})
    df = df.groupby(by=[attr, "date"]).agg(
        sales=pd.NamedAgg(column="sales", aggfunc="sum")
    ).reset_index()
    return df, df_dates

### Test the sales

In [182]:
df_pivot = df.pivot(index="date", columns=["attr"], values=["sales"])
df_pivot.columns = df_pivot.columns.droplevel(0)

In [193]:
df_pivot["brown"]

date
2016-12-01    0.000805
2016-12-02    0.000805
2016-12-03    0.001073
2016-12-04    0.001073
2016-12-05    0.017304
                ...   
2020-03-07    0.000805
2020-03-08    0.000805
2020-03-09    0.000805
2020-03-10    0.000805
2020-03-11    0.000805
Name: brown, Length: 1197, dtype: float64